In [1]:
import numpy as np
import os
import pydicom
import SimpleITK as sitk 
import re
from enum import Enum
from dicom_to_cnn.enums.TagEnum import *
from dicom_to_cnn.enums.SopClassUID import *
from dicom_to_cnn.tools.cleaning_dicom.cleaning_series import *
from dicom_to_cnn.tools.cleaning_dicom.folders import *
from dicom_to_cnn.model.reader.Instance import Instance
from dicom_to_cnn.model.reader.Series import Series
from dicom_to_cnn.model.reader.SeriesPT import SeriesPT

In [2]:
class TagCT(Enum):
    ConvolutionKernel = (0x18,0x1210)#('0018', '1210')

In [3]:
TagCT.ConvolutionKernel.name

'ConvolutionKernel'

In [4]:
TagCT.ConvolutionKernel.value

(24, 4624)

In [5]:
class SubInstance(Instance):
    def __init__(self, path:str, load_image:bool=True):
        super().__init__(path, load_image)

    
    def get_series_tags(self) -> dict :
        """method to gather series tags 

        Returns:
            [dict]: [dictionnary of every series tags and value]
        """
        series_tags={}
        for tag_address in TagsSeries:
            if tag_address.value in self.dicomData :
                series_tags[tag_address.name] = self.dicomData[tag_address.value].value
            else : series_tags[tag_address.name] = "Undefined"
       
        series_tags[TagCT.ConvolutionKernel.name] = self.get_ConvolutionKernel()
        return series_tags
    
    def get_ConvolutionKernel(self):
    
        if TagCT.ConvolutionKernel.value in [key for key in self.dicomData.keys()]: #[(f'{key.group:04}',(f'{key.elem:04}')) for key in self.dicomData.keys()]: 
            return self.dicomData[TagCT.ConvolutionKernel.value].value
        else: return 'Undefined'

    def display_dicom_metadata(self):
        print(self.dicomData)
    

In [6]:
class SubSeries(Series):
    def __init__(self,path):
        super().__init__(path)
    

    def get_first_instance_metadata(self) -> SubInstance:
        """method to read the first dicom instance in the folder

        Returns:
            [Instance]: [return Instance object]
        """
        firstFileName = self.file_names[0]
        return SubInstance(os.path.join(self.path,firstFileName), load_image=True)

    def get_series_details(self) -> dict:
        """Read the first dicom in the folder and store Patient / Study / Series
        informations

        Returns:
            [dict] -- [Return the details of a Serie from the first Dicom]
        """
        self.series_details = {}
        self.patient_details = {}
        self.study_details = {}

        dicomInstance = self.get_first_instance_metadata()
        

        self.series_details = dicomInstance.get_series_tags()
        self.patient_details = dicomInstance.get_patients_tags()
        self.study_details = dicomInstance.get_studies_tags()
        self.instance_details = dicomInstance.get_instance_tags()
        self.sop_class_uid = dicomInstance.get_sop_class_uid()
        self.is_image_series = dicomInstance.is_image_modality()
        self.ConvolutionKernel = dicomInstance.get_ConvolutionKernel()
        

        return {
            'series' : self.series_details,
            'study' : self.study_details,
            'patient' : self.patient_details,
            'path' : self.path,
            'files' : self.number_of_files,
            'instance' : self.instance_details,
            'Convolution Kernel' : self.ConvolutionKernel
        }

    def get_dcm_metadata(self):
        return self.get_first_instance_metadata()#.display_dicom_metadata()


In [7]:
class SeriesCT(SubSeries):
    """Get Series CT Nifti in 16 Bits

    Arguments:
        Series {String} -- Series Location Path
    """

    def __init__(self,path:str):
        """constructor

        Args:
            path (str): [path folder of CT serie ]
        """
        super().__init__(path)

    def get_numpy_array(self) -> np.ndarray:
        numpy_array = super().get_numpy_array()
        return numpy_array.astype(np.int16)

    def export_nifti(self, file_path:str):
        """method to export/save ndarray of series to nifti format

        Args:
            file_path (str): [directory+filename of the nifti]
        """
        sitk_img = sitk.GetImageFromArray( np.transpose(self.get_numpy_array(), (2,0,1) ))
        sitk_img = sitk.Cast(sitk_img, sitk.sitkInt16)
        original_pixel_spacing = self.instance_array[0].get_pixel_spacing()        
        original_direction = self.instance_array[0].get_image_orientation()
        try:
            sitk_img.SetDirection( (float(original_direction[0]), float(original_direction[1]), float(original_direction[2]), 
                                    float(original_direction[3]), float(original_direction[4]), float(original_direction[5]), 
                                    0.0, 0.0, 1.0) )
        except: 
            sitk_img.SetDirection( (float(original_direction[0]), float(original_direction[1]), float(original_direction[2]),  
                                    0.0, 1.0, 0.0,
                                    float(original_direction[3]), float(original_direction[4]), float(original_direction[5])) )
        sitk_img.SetOrigin( self.instance_array[0].get_image_position() )
        sitk_img.SetSpacing((original_pixel_spacing[0], original_pixel_spacing[1], self.get_z_spacing()))
        sitk.WriteImage(sitk_img, file_path)

In [8]:
def get_series_object(path:str):
    """
    class method to generate a Series object

    Args:
        path (str): [path folder of series]

    Returns:
        [Series]: [return a Series object]
    """
    try:
        first_file_name = os.listdir(path)[0]

    except Exception as err:
        print('ERRR')
        print(err)
        print(path)
    first_instance = SubInstance( os.path.join(path,first_file_name) )
    sop_class_uid = first_instance.get_sop_class_uid()
    if(sop_class_uid == ImageModalitiesSOPClass.PT.value or sop_class_uid == ImageModalitiesSOPClass.EnhancedPT.value):
        return SeriesPT(path)
    if (sop_class_uid == ImageModalitiesSOPClass.CT.value or sop_class_uid == ImageModalitiesSOPClass.EnhancedCT.value):
        return SeriesCT(path)
    else : return SubSeries(path)

In [9]:
def get_numbers(test_string):
   
    if isinstance(test_string,pydicom.multival.MultiValue):
        test_string=list(test_string)[0]
    temp = re.findall(r'\d+', test_string)
    res = list(map(int, temp))
    
    return res[0]

In [10]:
def atoi(text):
    if isinstance(text,pydicom.multival.MultiValue):
        text=list(text)[0]
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [11]:
Patient_dir ='/Users/nguyenkhaclan/Desktop/CT/2patients'#'/media/nguyen-k/A75D-9A3A/CASSIOPET/CT/CASSIOPET' #'/media/nguyen-k/BackupPlus/CT/2patients'
list_patient_id = os.listdir(Patient_dir)
dcm_folder = 'Screening_(Baseline)'

In [12]:
export_folder = '/Users/nguyenkhaclan/Desktop/CT/Json_Dir'
list_files = os.listdir(export_folder)
export_merged_folder='/Users/nguyenkhaclan/Desktop/CT/Json_Merged_Dir'
merged_file_name = 'merged_file'

os.system('rm -rf ' + export_folder + '/*')
os.system('rm -rf ' + export_merged_folder + '/*')

for patient_id in list_patient_id:

    print(patient_id)

    if '.DS_Store' in list_files : list_files.remove('.DS_Store'); 'Removing .DS_Store file...'

    path = os.path.join(Patient_dir,patient_id,dcm_folder);  'path of dicom directory Screening_(Baseline)...'

    
    serie_paths = get_series_path(path); 'paths of all dicom folders : '


    for serie_path in serie_paths:
        dicom_serie = get_series_object(serie_path)
        
        if isinstance(dicom_serie,SeriesCT):
            print('========================================================================================')
            
            #print(dicom_serie.get_dcm_metadata().dicomData[TagCT.ConvolutionKernel.value].value)
            #print(dicom_serie.get_dcm_metadata().dicomData)
            dicomsInfo = dicom_serie.get_series_details()
            
            if dicomsInfo['series']['Modality'] == 'CT':
                
                write_json_file(export_folder, dicomsInfo['series']['SeriesInstanceUID'], dicomsInfo); 'Write dicom info...'
                content_map = generate_merged_file(export_folder)
                write_json_file(export_merged_folder,merged_file_name , dict(content_map))
            

        
json_merged_file_path = os.path.join(export_merged_folder,merged_file_name + '.json')
print(json_merged_file_path)
list_path_non_intersting = find_non_intersting_series(json_merged_file_path); 'Get list path non intersting...'

031-23
/Users/nguyenkhaclan/Desktop/CT/2patients/031-23/Screening_(Baseline)/1.3.12.2.1107.5.1.4.11061.30000016112407592437400013703
/Users/nguyenkhaclan/Desktop/CT/2patients/031-23/Screening_(Baseline)/1.3.12.2.1107.5.1.4.11061.30000016112408011040300020187
/Users/nguyenkhaclan/Desktop/CT/2patients/031-23/Screening_(Baseline)/1.3.12.2.1107.5.1.4.11061.30000016112408011040300021151
/Users/nguyenkhaclan/Desktop/CT/2patients/031-23/Screening_(Baseline)/1.3.12.2.1107.5.1.4.11061.30000016112407592437400014193
/Users/nguyenkhaclan/Desktop/CT/Json_Dir/1.3.12.2.1107.5.1.4.11061.30000016112407592437400013703.json
/Users/nguyenkhaclan/Desktop/CT/Json_Dir/1.3.12.2.1107.5.1.4.11061.30000016112407592437400013703.json
/Users/nguyenkhaclan/Desktop/CT/Json_Dir/1.3.12.2.1107.5.1.4.11061.30000016112407592437400014193.json
009-26
/Users/nguyenkhaclan/Desktop/CT/2patients/009-26/Screening_(Baseline)/1.2.840.113619.2.290.3.1141511208.88.1484633698.566.3
/Users/nguyenkhaclan/Desktop/CT/2patients/009-26/Scr

'Get list path non intersting...'

In [13]:
print(list_path_non_intersting)

[]


In [14]:
with open(os.path.join(export_merged_folder,'merged_file.json')) as json_file:
    merged_data = json.load(json_file)
    list_merged_id = [key for key in merged_data.keys()]
    print(list_merged_id)

['031-23', '009-26']


In [15]:
#'SORT FILTRES'
list_StudyInstanceUID = []
list_CT_folders_selelected = []
os.system('rm -rf ' + export_folder + '/*')
serie_paths=[]
non_interesting_part = ['LUNG','POUMON','BONE']
list_conv_is_list = {'patient_id' : None, 'ConvKernel' : None}
for patient_id in list_merged_id:

    if '.DS_Store' in list_patient_id : list_patient_id.remove('.DS_Store')

    print(patient_id)

    
    SIUID = [k for k in merged_data[patient_id].keys()]

    for siuid in SIUID:
        print('=============')
        print(siuid)
        print('=============')
        SeIUID = [s for s in merged_data[patient_id][siuid]['Series'].keys()]
        print('SeIUID = ',SeIUID)
        serie_paths=[]
        for seiud in SeIUID:
            serie_path = merged_data[patient_id][siuid]['Series'][seiud]['path']
            serie_paths.append(serie_path) 

        print('serie_paths',serie_paths)

        
        list_CT_folders = []
        ListConvolutionKernel = []
        ListDicomInfo = []
        

        for serie_path in serie_paths:
            
            if serie_path not in list_path_non_intersting:
                #ListConvolutionKernel = []

                dicom_serie = get_series_object(serie_path)
                dicomsInfo = dicom_serie.get_series_details()
                list_CT_folders.append(serie_path)
                ListDicomInfo.append(dicomsInfo)
                ListConvolutionKernel.append(dicomsInfo['series']['ConvolutionKernel'])
        
        print('list_CT_folders',list_CT_folders)
        print('ListConvolutionKernel ini',ListConvolutionKernel)

        
        if len(ListConvolutionKernel)>1:
            
            if ListConvolutionKernel == ['STANDARD','LUNG'] or ListConvolutionKernel == ['LUNG', 'STANDARD', 'BONE'] or ListConvolutionKernel == ['LUNG', 'BONE']:
                dicomsInfoSelected = ListDicomInfo.copy()
                print('List Conv Selected ',ListConvolutionKernel)
                list_CT_folders_selelected.extend(list_CT_folders)
                
            
                    
            else:
                assert len(ListConvolutionKernel)==2
                
                if all(isinstance(conv, pydicom.multival.MultiValue) for conv in ListConvolutionKernel):
                    list_conv_is_list['patient_id'] = patient_id
                    list_conv_is_list['ConvKernel'] = ListConvolutionKernel
                    for i in range(len(ListConvolutionKernel)):
                        ListConvolutionKernel[i] = ListConvolutionKernel[i][0]
                    print('list of conv kernel list : ',ListConvolutionKernel) 

                for i in range(len(ListConvolutionKernel)):
                    if isinstance(ListConvolutionKernel[i],list):
                        ListConvolutionKernel[i] = ListConvolutionKernel[i][0] 

                splitstr=[]
                
                for i in range(len(ListConvolutionKernel)):
                    splitstr.append(natural_keys(ListConvolutionKernel[i]))

                print('splitstr : ',splitstr)
                
                

                max_tmp = splitstr[0][0]
                max_index = [0]
                for conv_id in range(1,2): # only 2 CT piles
                    print(f'max_index {conv_id}',max_index)
                    if splitstr[conv_id][0] < max_tmp:
                        print('max')
                        max_tmp = splitstr[conv_id][0]
                        max_index = [conv_id]
                        print(f'max_index{conv_id}',max_index)
                        
                    elif splitstr[conv_id][0] == max_tmp: 
                        print('equal')
                        max_tmp = splitstr[conv_id][0]
                        max_index.append(conv_id)
                    else:
                        print('continue...')

                    print(f'max_index {conv_id}',max_index)
                    print('len max_index  > 1 ',len(max_index)>1)

                if len(max_index)>1:
                    print('KO.......')
                    print(len(max_index))
                    if len(splitstr[max_index[0]]) > 1:
                        max_tmp_1 = splitstr[max_index[0]][1]
                        max_index_1 = max_index.copy()
                        for conv_id in range(1,2):
                            if splitstr[conv_id][1] > max_tmp_1:
                                max_tmp_1 = splitstr[conv_id][1]
                                max_index_1 = [conv_id]
                                print(f'max_index_1 {conv_id}',max_index_1)

                            elif splitstr[conv_id][1] == max_tmp: 
                                max_tmp_1 = splitstr[conv_id][1]
                                max_index_1.append(conv_id)
                            else:
                                print('continue...')
                
                if "max_index_1" in locals():      
                    if len(max_index_1)>1:
                        max_tmp_2 = splitstr[max_index[0]][2]
                        max_index_2 = max_index.copy()
                        for conv_id in range(1,2):
                            if splitstr[conv_id][2] < max_tmp_2:
                                max_tmp_2 = splitstr[conv_id][2]
                                max_index_2 = [conv_id]
                                print(f'max_index_2 {conv_id}',max_index_2)

                            elif splitstr[conv_id][2] == max_tmp: 
                                max_tmp_2 = splitstr[conv_id][2]
                                max_index_2.append(conv_id)
                            else:
                                print('continue...')
                
                if "max_index_2" in locals():
                    index = max_index_2[0]
                elif  "max_index_1" in locals():
                    index = max_index_1[0]
                else:
                    if any(conv=='STANDARD' for conv in ListConvolutionKernel):
                        index = max_index
                    else:
                        index = max_index[0]

                        
                print('index : ',index)
                if isinstance(index,list):
                    dicomsInfoSelected = [ListDicomInfo[ind] for ind in index]
                    for ind in index:
                        list_CT_folders_selelected.append(list_CT_folders[ind])
                        remove_bi_file(serie_paths[ind])
                    print('ListConvolutionKernel',ListConvolutionKernel)
                    print('List Conv Selected',[ListConvolutionKernel[ind] for ind in index])
                else:
                    
                    dicomsInfoSelected = ListDicomInfo[index]
                    list_CT_folders_selelected.append(list_CT_folders[index])

                    print('ListConvolutionKernel',ListConvolutionKernel)
                    print('List Conv Selected',ListConvolutionKernel[index])

                    remove_bi_file(serie_paths[index])
                
                
                
                print('Selecting dicom info done...')

                if "max_index_1" in locals() : del max_index_1
                if "max_index_2" in locals() : del max_index_2
        elif len(ListConvolutionKernel)==1:
            dicomsInfoSelected = ListDicomInfo[0]
            list_CT_folders_selelected.append(list_CT_folders[0])
            print('ListConvolutionKernel',ListConvolutionKernel)
            print('List Conv Selected',ListConvolutionKernel[0])
            
        else:
            continue

        if isinstance(dicomsInfoSelected,list): 
            for i in range(len(dicomsInfoSelected)):
                write_json_file(export_folder, dicomsInfoSelected[i]['series']['SeriesInstanceUID'], dicomsInfoSelected)
            
        else:
            write_json_file(export_folder, dicomsInfoSelected['series']['SeriesInstanceUID'], dicomsInfoSelected)
        print('write json done...')

031-23
1.2.840.113711.3413733.456028.501674924.26.2116281012.10
SeIUID =  ['1.3.12.2.1107.5.1.4.11061.30000016112407592437400013703', '1.3.12.2.1107.5.1.4.11061.30000016112407592437400014193']
serie_paths ['/Users/nguyenkhaclan/Desktop/CT/2patients/031-23/Screening_(Baseline)/1.3.12.2.1107.5.1.4.11061.30000016112407592437400013703', '/Users/nguyenkhaclan/Desktop/CT/2patients/031-23/Screening_(Baseline)/1.3.12.2.1107.5.1.4.11061.30000016112407592437400014193']
list_CT_folders ['/Users/nguyenkhaclan/Desktop/CT/2patients/031-23/Screening_(Baseline)/1.3.12.2.1107.5.1.4.11061.30000016112407592437400013703', '/Users/nguyenkhaclan/Desktop/CT/2patients/031-23/Screening_(Baseline)/1.3.12.2.1107.5.1.4.11061.30000016112407592437400014193']
ListConvolutionKernel ini ['B30s', 'B70s']
splitstr :  [['B', 30, 's'], ['B', 70, 's']]
max_index 1 [0]
equal
max_index 1 [0, 1]
len max_index  > 1  True
KO.......
2
max_index_1 1 [1]
index :  1
ListConvolutionKernel ['B30s', 'B70s']
List Conv Selected B70s
Sel

In [16]:
print('list_CT_folders_selelected',list_CT_folders_selelected)

list_CT_folders_selelected ['/Users/nguyenkhaclan/Desktop/CT/2patients/031-23/Screening_(Baseline)/1.3.12.2.1107.5.1.4.11061.30000016112407592437400014193', '/Users/nguyenkhaclan/Desktop/CT/2patients/009-26/Screening_(Baseline)/1.2.840.113619.2.290.3.1141511208.88.1484633698.566.3']


In [17]:
list_id = []
for path in list_CT_folders_selelected:
     list_id.append(path.split('/')[6])

dup_index = [idx for idx, item in enumerate(list_id) if item in list_id[:idx]]
print(dup_index)

[]


In [18]:
save_nifti_dir = '/home/nguyen-k/Bureau/segCassiopet/CT/nifti_final'#'/media/nguyen-k/A75D-9A3A/CASSIOPET/CT/nifti'
try: 
    os.mkdir(save_nifti_dir) 
except OSError as error: 
    print(error)  
os.system('rm -rf ' + save_nifti_dir + '/*')

[Errno 2] No such file or directory: '/home/nguyen-k/Bureau/segCassiopet/CT/nifti_final'


0

In [19]:
list_patient_no_img = []

for i in range(len(list_CT_folders_selelected)):
   
    CT_path = list_CT_folders_selelected[i]
    print(CT_path) 
    patient_id = CT_path.split('/')[6]
    patient_id_nifti_dir = os.path.join(save_nifti_dir,patient_id)
    print(patient_id_nifti_dir)
    try: 
        os.mkdir(patient_id_nifti_dir) 
    except OSError as error: 
        print(error)  

    CT = SeriesCT(CT_path)
    try:
        if i not in dup_index:
            CT.export_nifti(os.path.join(patient_id_nifti_dir,'patientCT.nii'))
        else:
            CT.export_nifti(os.path.join(patient_id_nifti_dir,'patientCT_'+str(i)+'.nii'))
    except:
        print(f'Patient {patient_id} has only one z position...')
        list_patient_no_img.append(patient_id)
    

/Users/nguyenkhaclan/Desktop/CT/2patients/031-23/Screening_(Baseline)/1.3.12.2.1107.5.1.4.11061.30000016112407592437400014193
/home/nguyen-k/Bureau/segCassiopet/CT/nifti_final/031-23
[Errno 2] No such file or directory: '/home/nguyen-k/Bureau/segCassiopet/CT/nifti_final/031-23'


** ERROR (nifti_image_write_hdr_img2): cannot open output file '/home/nguyen-k/Bureau/segCassiopet/CT/nifti_final/031-23/patientCT.nii'


/Users/nguyenkhaclan/Desktop/CT/2patients/009-26/Screening_(Baseline)/1.2.840.113619.2.290.3.1141511208.88.1484633698.566.3
/home/nguyen-k/Bureau/segCassiopet/CT/nifti_final/009-26
[Errno 2] No such file or directory: '/home/nguyen-k/Bureau/segCassiopet/CT/nifti_final/009-26'
Patient 009-26 has only one z position...


In [20]:
print(list_patient_no_img)

['009-26']
